# Signal Classifier

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf
#np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
def plot(fig,x,y,name):
    fig.add_trace(go.Scatter(x=x,y=y,mode='lines',name=name))


## Defintion of Different Signal Functions

In [2]:
def dft(N,M=40):
    #w=np.e**((2*np.pi*-1j)/N)
    power=np.zeros((M,N),dtype=complex)
    xj = np.linspace(-np.pi, np.pi, N)
    for i in range(M):
        for j in range(N):
            #power[i][j]=((-N/2)+i)*j
            power[i][j] = -1j*(i-M/2)*xj[j]
    #DFT=(w**power)/N
    DFT = np.e**power
    return DFT
def box(x,a=2,b=5,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k))
        fkhat[k==0]=(2*a*b)
        fkhat[k!=0]=((2*b*np.sin(a*k[k!=0]))/k[k!=0])
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(box(x,a,b,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=b
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(box(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def saw(x,a=2,b=5,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=0
        fkhat[k!=0]=(2*1j*b*(np.sin(a*k[k!=0])-a*k[k!=0]*np.cos(a*k[k!=0])))/(k[k!=0]**2)
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(saw(x,a,b,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=-b*x[np.abs(x)<a]
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(saw(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def exp(x,a=2,b=2,c=-1,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==0]=(2*((a*b*c)+np.sinh(a*b)))/b
        fkhat[k!=0]=((2*c*np.sin(a*k[k!=0]))/k[k!=0])+((2*np.sinh(a*(b+(1j*k[k!=0]))))/(b+(1j*k[k!=0])))
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(exp(x,a,b,c,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c+np.e**(-b*x[[np.abs(x)<a]])
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
           return y/np.max(abs(exp(x,a,b,c,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def sinu(x,a=2,b=2,c=-1,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40):
    if fourier:
        k=np.linspace(int((-N/2)),int((N/2)),N+1).astype(complex)
        fkhat=np.zeros(len(k)).astype(complex)
        fkhat[k==b]=((1j*c*np.sin(2*a*b))/(2*b))-(1j*a*c)
        fkhat[k==-b]=(1/2)*1j*c*((2*a)-((np.sin(2*a*b))/b))
        fkhat[k!=b]=(2*1j*c*((b*np.cos(a*b)*np.sin(a*k[k!=b]))-(k[k!=b]*np.sin(a*b)*np.cos(a*k[k!=b]))))/((b**2)-(k[k!=b]**2))
        y=fkhat/(2*np.pi)
        if normalized:
            return y/np.max(abs(sinu(x,a,b,c,normalized=False)))
        return y
    y=np.zeros(len(x))
    y[np.abs(x)<a]=c*np.sin(b*x[np.abs(x)<a])
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(sinu(x,a,b,c,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def gaus(x,a=2,b=2,normalized=True,noise=False,noiseParameter=0.1,fourier=False,N=40,M=40):
    if fourier:
        fkhat=((np.dot(dft(N,M),gaus(x,a,b)))/N)
        if normalized:
            return fkhat/np.max(abs(gaus(x,a,b,normalized=False)))
        return fkhat
    y=np.zeros(len(x))
    y[np.abs(x)<a]=np.e**(-a*(x[np.abs(x)<a]**(2*b)))
    y=np.nan_to_num(y)
    if noise:
        y=y+np.random.normal(0,noiseParameter,len(x))
        if normalized:
            return y/np.max(abs(gaus(x,a,b,normalized=False)))
        return y
    if normalized:
        return y/np.max(abs(y))
    return y
def FourierSeries(cn,X,N):
    fx=[]
    '''
    Cn=np.zeros(len(cn))
    Cn[0]=cn[int(N/2)]
    Cn[1:int(N/2)-1]=cn[1:int(N/2)-1]
    Cn[int(N/2)+1:N]=cn[int(N/2)+1:N]
    fx=np.fft.ifft(Cn)
    '''
    
    for x in X:
        result=0
        for i in range(int((-N/2)),int((N/2))):
            result=result+cn[int(i+(N/2))]*(np.e**(1j*i*x)) #Calculate Fourier Series approximation using formula
        fx.append(result.real)
    return fx

In [3]:
SignalsDict={'Box':1,'Saw':2,'Exp':3,'Sin':4,'Gaus':5}
def Generate(signal,x,Amount,normalized=True,fourier=False,N=40,M=40,Noise=False,noiseParameter=0.1):
    signalOutput=[]
    length=Amount+len(x)
    print(SignalsDict[signal])
    if(SignalsDict[signal]==1):
        #a=np.linspace(np.pi/4,np.pi/2,len(x))
        a=np.linspace(.10,2.90,length)
        b=np.append(np.linspace(-100,-0.01,int(length/2)),np.linspace(0.01,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=box(x,a[i],b[i],normalized=normalized,fourier=True,N=N)
                normalization=1#np.max(abs(F))
                #BoxSignal.append(F/normalization)
                #BoxSignalFourier.append(fx)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=box(x,a[i],b[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=box(x,a[i],b[i],normalized=normalized)
                #normalization=1#np.max(abs(F))
                #BoxSignal.append(F/normalization)
                #BoxSignalFourier.append(fx)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
    elif(SignalsDict[signal]==2):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-100,-0.01,int(length/2)),np.linspace(0.01,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=saw(x,a[i],b[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=saw(x,a[i],b[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=saw(x,a[i],b[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
    elif(SignalsDict[signal]==3):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-1,-0.1,int(length/2)),np.linspace(0.1,1,int(length/2)))
        c=np.append(np.linspace(-3,-1.01,int(length/2)),np.linspace(-1.01,1,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        c=np.random.permutation(c)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=exp(x,a[i],b[i],c[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=exp(x,a[i],b[i],c[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        else:
            for i in range(Amount):
                F=exp(x,a[i],b[i],c[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
    elif(SignalsDict[signal]==4):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-2*np.pi,-0.3,int(length/2)),np.linspace(0.3,2*np.pi,int(length/2)))
        c=np.append(np.linspace(-100,-0.1,int(length/2)),np.linspace(0.1,100,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)
        c=np.random.permutation(c)
        if fourier:
            print(a[0])
            for i in range(Amount):
                cn=sinu(x,a[i],b[i],c[i],normalized=normalized
                ,fourier=True,N=N)
                signalOutput.append(FourierSeries(cn,x,N))
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=sinu(x,a[i],b[i],c[i],noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
        else:
            for i in range(Amount):
                F=sinu(x,a[i],b[i],c[i],normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount],c[0:Amount]]
    elif(SignalsDict[signal]==5):
        a=np.linspace(np.pi/4,np.pi/2,length)
        b=np.append(np.linspace(-10,-1,int(length/2)),np.linspace(1,10,int(length/2)))
        a=np.random.permutation(a)
        b=np.random.permutation(b)

        if fourier:
            print(a[0])
            for i in range(Amount):
                x=np.linspace(-np.pi,np.pi,N)
                cn=gaus(x,a[i],int(b[i]),normalized=normalized,fourier=True,N=N,M=M)
                signalOutput.append(FourierSeries(cn,x,M))
            return [signalOutput,a[0:Amount],b[0:Amount]]
        elif Noise:
            for i in range(Amount):
                F=gaus(x,a[i],int(b[i]),noise=True,normalized=normalized,noiseParameter=noiseParameter)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]
        else:
            for i in range(Amount):
                F=gaus(x,a[i],int(b[i]),normalized=normalized)
                signalOutput.append(F)
            return [signalOutput,a[0:Amount],b[0:Amount]]

## Testing
###

In [8]:
x=np.linspace(-np.pi,np.pi,100)

BoxSignal=Generate('Box',x,Amount=1000)
SawSignal=Generate('Saw',x,Amount=1000)
ExpSignal=Generate('Exp',x,Amount=1000)
SinSignal=Generate('Sin',x,Amount=1000)
GausSignal=Generate('Gaus',x,Amount=1000)

#parameters




1
2
3
4
5


In [70]:
np.shape(BoxSignal)

(100, 100)

In [11]:
Signals=[BoxSignal[0],SawSignal[0],ExpSignal[0],SinSignal[0],GausSignal[0]]
SignalLabels=[
    1*np.ones(1000),
    2*np.ones(1000),
    3*np.ones(1000),
    4*np.ones(1000),
    5*np.ones(1000)]
SignalsTrain=np.array(np.concatenate(Signals).flat)
SignalsTrain=SignalsTrain.reshape((5000,100))
SignalLabelsTrain=np.array(np.concatenate(SignalLabels).flat)

In [130]:
np.shape(SignalsTrain)

(500, 100)

In [12]:
modelNN = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6)
])
modelNN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
modelNN.fit(SignalsTrain, SignalLabelsTrain, epochs=100)


Epoch 1/100
157/157 [==============================] - 1s 2ms/step - loss: 0.8703 - accuracy: 0.6884
Epoch 2/100
157/157 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.8552
Epoch 3/100
157/157 [==============================] - 0s 2ms/step - loss: 0.3483 - accuracy: 0.8858
Epoch 4/100
157/157 [==============================] - 0s 2ms/step - loss: 0.2924 - accuracy: 0.9056
Epoch 5/100
157/157 [==============================] - 0s 2ms/step - loss: 0.2503 - accuracy: 0.9232
Epoch 6/100
157/157 [==============================] - 0s 3ms/step - loss: 0.2250 - accuracy: 0.9288
Epoch 7/100
157/157 [==============================] - 0s 3ms/step - loss: 0.2080 - accuracy: 0.9340
Epoch 8/100
157/157 [==============================] - 0s 3ms/step - loss: 0.1954 - accuracy: 0.9382
Epoch 9/100
157/157 [==============================] - 0s 3ms/step - loss: 0.1781 - accuracy: 0.9426
Epoch 10/100
157/157 [==============================] - 0s 3ms/step - loss: 0.1677 - accura

In [13]:

BoxSignal=Generate('Box',x,Amount=1000)
SawSignal=Generate('Saw',x,Amount=1000)
ExpSignal=Generate('Exp',x,Amount=1000)
SinSignal=Generate('Sin',x,Amount=1000)
GausSignal=Generate('Gaus',x,Amount=1000)
a=[BoxSignal[1],SawSignal[1],ExpSignal[1],SinSignal[1],GausSignal[1]]
b=[BoxSignal[2],SawSignal[2],ExpSignal[2],SinSignal[2],GausSignal[2]]
c=[np.zeros(1000),np.zeros(1000),ExpSignal[3],SinSignal[3],np.zeros(1000)]
Signals=[BoxSignal[0],SawSignal[0],ExpSignal[0],SinSignal[0],GausSignal[0]]
SignalLabels=[
    1*np.ones(1000),
    2*np.ones(1000),
    3*np.ones(1000),
    4*np.ones(1000),
    5*np.ones(1000)]
SignalsTest=np.array(np.concatenate(Signals).flat)
a=np.array(np.concatenate(a).flat)
b=np.array(np.concatenate(b).flat)
c=np.array(np.concatenate(c).flat)

SignalsTest=SignalsTest.reshape((5000,100))
SignalLabelsTest=np.array(np.concatenate(SignalLabels).flat)

1
2
3
4
5


In [14]:
test_loss, woNoiseAcc = modelNN.evaluate(SignalsTest,  SignalLabelsTest, verbose=2,use_multiprocessing=True,)
#modelNN.evaluate()
#modelNN.predict_batch(BoxSignal[0])
probability_model=tf.keras.Sequential([modelNN,tf.keras.layers.Softmax()])
predictions=probability_model.predict(SignalsTest)

157/157 - 1s - loss: 0.0605 - accuracy: 0.9784


In [15]:
testResults=[]
for i in range(len(predictions)):
    a=np.max(predictions[i])
    maxElement=np.where(predictions[i]==a)
    testResults.append(maxElement[0][0])


In [149]:
len(testResults)

5000

In [19]:
Model2Summ=pd.DataFrame()
Model2Summ['Actual Classifcation']=SignalLabelsTest
Model2Summ['Predicted Classification']=testResults
Model2Summ['a']=a
Model2Summ['b']=b
Model2Summ['c']=c


In [20]:
Model2Summ.to_csv('Model2Results.csv')

In [21]:
modelNN.save('Model2TrainedTF')

INFO:tensorflow:Assets written to: Model2TrainedTF/assets


In [22]:
ls

 archive/                         Model2.ipynb       '#Report.lyx#'
 ClassifyingSignals.ipynb         Model2Results.csv   Report.lyx
 ClassifyingSignalsModel2.ipynb   Model2TrainedTF/    Report.lyx~
 FourierSeries.ipynb              Model3.ipynb        requirements.txt
 Model1.ipynb                     Model3Results.csv   SignalGeneration.ipynb
 Model1Results.csv                Model3TrainedTF/    venv/
 Model1TrainedTF/                 README.md
